In [1]:
# %pip install matplotlib

In [13]:
#!/usr/bin/python3
# -*- coding:utf-8 -*-

import itertools
import os
import pathlib
import random
import sys
import cv2
import numpy as np
from matplotlib import pyplot as plt, image as mpimg
import pandas as pd
from difficulty_levels import DifficultyLevels
from tensorflow import keras
import tensorflow as tf
from tensorflow.python.keras.callbacks import ModelCheckpoint


download_folder = "tracking_data_download"
labeled_images_folder = "labeled_images"

RANDOM_SEED = 42
NUMBER_OF_CLASSES = 3

results_folder = "ml_results"
data_folder_path = os.path.join("..", "post_processing", download_folder)
# print(data_folder_path)

NEW_IMAGE_SIZE = 128

In [3]:
def set_random_seed(seed=RANDOM_SEED):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [4]:
def show_result_plot(train_history, epochs, metric="categorical_accuracy", output_folder=results_folder,
                     output_name="train_history.png"):

    acc = train_history.history[f"{metric}"]
    val_acc = train_history.history[f"val_{metric}"]
    loss = train_history.history["loss"]
    val_loss = train_history.history["val_loss"]

    epochs_range = range(epochs)
    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')

    # save plot to file and show in a new window
    plt.savefig(os.path.join(output_folder, output_name))
    plt.show()

In [12]:
def get_participant_images(participant_folder, use_folder=False, use_step_size=False):
    post_processing_folder_path = os.path.join("..", "post_processing")
    participant_folder_path = os.path.join(post_processing_folder_path, download_folder, participant_folder)

    # iterate over the csv file and yield the image paths and their corresponding difficulty level
    images_label_log = os.path.join(participant_folder_path, "labeled_images.csv")
    labeled_images_df = pd.read_csv(images_label_log)

    # FIXME unfortunately a different order changes the results :(
    # for difficulty_level in ["easy", "hard", "medium"]:
    for difficulty_level in labeled_images_df.difficulty.unique():
        # create a subset of the df that contains only the rows with this difficulty level
        sub_df = labeled_images_df[labeled_images_df.difficulty == difficulty_level]

        for idx, row in sub_df.iterrows():
            image_path = row["image_path"]
            full_image_path = os.path.join(post_processing_folder_path, image_path)
            # current_image = cv2.imread(full_image_path)
            yield full_image_path, difficulty_level

In [6]:
def preprocess_train_test_data(data):
    train_test_data = []

    for participant in data:
        for image_path, difficulty_level in get_participant_images(participant, use_folder=False, use_step_size=False):
            label_vector = DifficultyLevels.get_one_hot_encoding(difficulty_level)
            try:
                # grayscale_img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                # use this instead: (for reason see
                # https://stackoverflow.com/questions/37203970/opencv-grayscale-mode-vs-gray-color-conversion#comment103382641_37208336)
                color_img = cv2.imread(image_path)
                grayscale_img = cv2.cvtColor(color_img, cv2.COLOR_BGR2GRAY)

                resized_img = cv2.resize(grayscale_img, (NEW_IMAGE_SIZE, NEW_IMAGE_SIZE))
                train_test_data.append([np.array(resized_img, dtype=np.uint8), label_vector, image_path])

            except Exception as e:
                sys.stderr.write(f"\nError in reading and resizing image '{image_path}': {e}")

    # random.shuffle(train_test_data)
    return train_test_data

In [7]:
def start_preprocessing():
    set_random_seed(RANDOM_SEED)  # for reproducibility

    without_participants = ["participant_1", "participant_2", "participant_4", "participant_5", "participant_6",
                            "participant_7", "participant_8", "participant_9", "participant_11", "participant_12",
                            "participant_13"]

    all_participants = os.listdir(data_folder_path)
    # remove some participants for testing
    all_participants = [p for p in all_participants if p not in set(without_participants)]

    random.shuffle(all_participants)

    train_ratio = 0.8
    train_split = int(len(all_participants) * train_ratio)
    train_participants = all_participants[:train_split]
    test_participants = all_participants[train_split:]
    print(f"{len(train_participants)} participants used for training: {train_participants}")
    print(f"{len(test_participants)} participants used for validation: {test_participants}")

    train_data = preprocess_train_test_data(train_participants)
    test_data = preprocess_train_test_data(test_participants)
    print("Len training data: ", len(train_data))
    print("Len test data: ", len(test_data))

    # TODO save them all as one and split later when reading in?
    train_images = []  # features for training
    train_labels = []  # labels for training
    train_paths = []  # paths to images in train data
    for img_data, label, path in train_data:
        train_images.append(img_data)
        train_labels.append(label)
        train_paths.append(path)

    test_images = []  # features for testing
    test_labels = []  # labels for testing
    test_paths = []  # paths to images in test data
    for img_data, label, path in test_data:
        test_images.append(img_data)
        test_labels.append(label)
        test_paths.append(path)

    train_images = np.asarray(train_images).reshape(-1, NEW_IMAGE_SIZE, NEW_IMAGE_SIZE, 1)
    test_images = np.asarray(test_images).reshape(-1, NEW_IMAGE_SIZE, NEW_IMAGE_SIZE, 1)
    # normalize all images to [0, 1] for the neural network
    train_images = train_images.astype('float32') / 255.0
    test_images = test_images.astype('float32') / 255.0

    result_folder = "ml_results"
    if not os.path.exists(result_folder):
        os.mkdir(result_folder)

    # TODO use np.savez() to save compressed?
    np.save(os.path.join(result_folder, "train_images.npy"), train_images, allow_pickle=False)
    np.save(os.path.join(result_folder, "train_labels.npy"), train_labels, allow_pickle=False)
    np.save(os.path.join(result_folder, "train_paths.npy"), train_paths, allow_pickle=False)

    np.save(os.path.join(result_folder, "test_images.npy"), test_images, allow_pickle=False)
    np.save(os.path.join(result_folder, "test_labels.npy"), test_labels, allow_pickle=False)
    np.save(os.path.join(result_folder, "test_paths.npy"), test_paths, allow_pickle=False)
    
    # return (train_images, train_labels, train_paths), (test_images, test_labels, test_paths)

In [8]:
def show_imgs_with_prediction(image_paths, actual_labels, probabilities):
    plt.figure(figsize=(10, 10))
    num_images = 25
    # image_slice = random.sample(image_paths, num_images)

    for n in range(num_images):
        ax = plt.subplot(5, 5, n + 1)
        img = mpimg.imread(image_paths[n])
        plt.imshow(img)
        plt.axis('off')

        probability_vector = probabilities[n]
        highest_index = np.argmax(probability_vector)
        print(f"Probability Vector: {probability_vector}, highest index: {highest_index}")

        actual_label_vector = actual_labels[n]
        correct_label = None
        for label in DifficultyLevels.values():
            label_vector = DifficultyLevels.get_one_hot_encoding(label)
            if all(label_vector == actual_label_vector):
                correct_label = label
                break

        for label in DifficultyLevels.values():
            label_vector = DifficultyLevels.get_one_hot_encoding(label)
            if label_vector[highest_index]:
                plt.title(f"{probability_vector[highest_index] * 100:.0f}% {label} ({correct_label})")
                break

    plt.savefig(os.path.join(results_folder, 'result_classification.png'))


def test_model(test_data: tuple, model_path, checkpoint_folder_name):
    images_test, labels_test, paths_test = test_data

    if os.path.exists(model_path):
        loaded_model = keras.models.load_model(model_path)
        print("Model successfully loaded")

        prediction = loaded_model.predict(images_test)
        # print(f"Prediction result: {prediction}")
        show_imgs_with_prediction(paths_test, labels_test, prediction)

        test_loss, test_acc = loaded_model.evaluate(images_test, labels_test, verbose=1)
        print("Test accuracy: ", test_acc * 100)

        # load latest (i.e. the best) checkpoint
        """
        loaded_model = keras.models.load_model(model_path)  # re-create the model first!
        checkpoint_folder = os.path.join(results_folder, checkpoint_folder_name)
        latest = tf.train.latest_checkpoint(checkpoint_folder)
        loaded_model.load_weights(latest)

        # and re-evaluate the model
        loss, acc = loaded_model.evaluate(images_test, labels_test, verbose=1)
        print(f"Accuracy with restored model weights: {100 * acc:5.2f}%")
        """
    else:
        sys.stderr.write("No saved model found!")

In [9]:
def build_model_sequential(input_shape, num_classes=NUMBER_OF_CLASSES):
    model = keras.Sequential(
        [
            keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
            keras.layers.MaxPooling2D(pool_size=(2, 2)),
            keras.layers.Dropout(0.25),

            keras.layers.Conv2D(64, (3, 3), activation='relu'),
            keras.layers.MaxPooling2D(pool_size=(2, 2)),
            keras.layers.Dropout(0.25),

            keras.layers.Conv2D(128, (3, 3), activation='relu'),
            keras.layers.MaxPooling2D(pool_size=(2, 2)),
            keras.layers.Dropout(0.25),

            keras.layers.Flatten(),
            # units in the last layer should be a power of two (e.g. 64, 128, 512, 1024)
            keras.layers.Dense(units=1024, activation="relu"),
            keras.layers.Dropout(0.5),

            # units=3 as we have 3 classes -> we need a vector that looks like this: [0.2, 0.5, 0.3]
            keras.layers.Dense(units=num_classes, activation="softmax")  # softmax for multi-class classification, see
            # https://medium.com/deep-learning-with-keras/how-to-solve-classification-problems-in-deep-learning-with-tensorflow-keras-6e39c5b09501
        ]
    )

    model.summary()
    # optimizer = SGD(lr=0.01, momentum=0.9)
    # other optimizers like "rmsprop" or "adamax" ?

    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"])
    return model

In [10]:
def train_model(train_data: tuple, test_data: tuple, model_path, checkpoint_path):
    images_train, labels_train, paths_train = train_data
    images_test, labels_test, paths_test = test_data

    # print(tf.config.list_physical_devices('GPU'))
    # print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

    print("Len train data:", len(images_train))
    print("Len test data:", len(images_test))

    set_random_seed()

    # image_shape = images_train.shape[1:]
    image_shape = (NEW_IMAGE_SIZE, NEW_IMAGE_SIZE, 1)
    model = build_model_sequential(input_shape=image_shape)

    EPOCHS = 32
    BATCH_SIZE = 32
    # VALIDATION_SPLIT = 0.25

    checkpoint_callback = ModelCheckpoint(checkpoint_path, monitor='val_categorical_accuracy', verbose=1, mode="max",
                                          save_best_only=True, save_weights_only=True)
    lr_callback = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5,
                                                    verbose=1)

    # history = model.fit(images_train, labels_train, batch_size=BATCH_SIZE, validation_split=VALIDATION_SPLIT,
    #                     verbose=1, epochs=EPOCHS, callbacks=[checkpoint_callback, lr_callback])

    history = model.fit(images_train, labels_train, batch_size=BATCH_SIZE, verbose=1, epochs=EPOCHS,
                        validation_data=(images_test, labels_test), callbacks=[checkpoint_callback, lr_callback])
    print(history.history)
    model.save(model_path)

    show_result_plot(history, EPOCHS, metric="categorical_accuracy", output_folder=results_folder)

In [11]:
def main(train=True, test=False):
    # TODO don't use pickle
    images_train = np.load(os.path.join(results_folder, 'train_images.npy'), allow_pickle=False)
    labels_train = np.load(os.path.join(results_folder, 'train_labels.npy'), allow_pickle=False)
    paths_train = np.load(os.path.join(results_folder, 'train_paths.npy'), allow_pickle=False)

    images_test = np.load(os.path.join(results_folder, 'test_images.npy'), allow_pickle=False)
    labels_test = np.load(os.path.join(results_folder, 'test_labels.npy'), allow_pickle=False)
    paths_test = np.load(os.path.join(results_folder, 'test_paths.npy'), allow_pickle=False)

    MODEL_NAME = 'Cognitive-Load-CNN-Model.h5'
    model_save_location = os.path.join(results_folder, MODEL_NAME)

    checkpoint_folder = "checkpoints"
    checkpoint_path = os.path.join(results_folder, checkpoint_folder,
                                   "checkpoint-improvement-{epoch:02d}-{val_categorical_accuracy:.3f}.ckpt")

    if train:
        train_model((images_train, labels_train, paths_train), (images_test, labels_test, paths_test),
                    model_save_location, checkpoint_path)

    if test:
        # TODO should not be the ones used for validation in train_model() !
        test_model((images_test, labels_test, paths_test), model_save_location, checkpoint_folder)

In [ ]:
start_preprocessing()

In [14]:
# train a machine learning model
main(train=True, test=True)

Len train data: 53056
Len test data: 20222
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_2 (Con

KeyboardInterrupt: 